In [3]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

In [71]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
encoder_out = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(utterance.shape)
print(utterance_mask.shape)
print(encoder_out.shape)
print(prev_output.shape)

torch.Size([78, 20])
torch.Size([78, 20])
torch.Size([78, 20, 768])
torch.Size([78, 19])


In [76]:
utterance_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')

In [23]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

torch.Size([1938])

In [97]:
import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.attention 
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.attention)
from nlptools.zoo.encoders.attention import MultiheadAttention, Attention
from nlptools.zoo.encoders.transformer import TransformerDecoder
embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 3072
dropout = 0
decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask)
output.shape

torch.Size([78, 19, 30522])